# Машинное обучение, часть 1, ШАД
## Домашнее задание 3: Логистическая регрессия



In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

...  # допиши необходимые импорты

sns.set(style="whitegrid", palette="Set2")

*Привет!*

*Перед тобой увлекательная домашка по линейной регрессии и градиентным методам оптимизации. Надеемся, что тебе она понравится, ты точно найдешь в ней что-то интересное. Конечно, просто не будет, но никто этого и не обещал. В условии оставлены некоторые скрытые подсказки, будет хорошо, если ты сначала постараешься подумать самостоятельно, а затем раскроешь содержимое подсказки. Если у тебя будут вопросы по условию, можешь обратиться с ними в чат. Только очень желательно не делиться в чате фрагментами решения.*

*Успехов в решении!*

---
### Задача 1.

Найдите оценку параметра $\theta$ методом максимального правдоподобия по выборке размера $n$ из распределения:
* $\mathrm{Exp}(\theta)$ &mdash; экспоненциальное, где $\theta>0$;
* $\mathrm{Pois}(\theta)$ &mdash; пуассоновское, где $\theta>0$.

---
### Задача 2.

*Для начала посмотрим на простых примерах, как работает логистическая регрессия, и поймем некоторые ее важные свойства.*

*Перед выполнением задачи ознакомтесь с ноутбуком по логистической регрессии с занятия.*

> Одно из интересных свойств модели логистической регрессии — *при соблюдении её предположений* она дает возможность получать **несмещенные оценки вероятностей** принадлежности объекта к определенному классу, близкие к идеально скалиброванным. Напомним, модель называется идеально скалиброванной в следующем случае. Рассмотрим объект $x$ и соответствующее предсказание вероятности $\widehat{p}(x)$ для класса 1. Если взять небольшую окрестность объекта $x$, то доля объектов класса 1 в этой окрестности будет приблизительно равна $\widehat{p}(x)$.  

Далее проверим это свойство на конкретных примерах.

С помощью кода ниже сгенерируйте данные, состоящие из одного вещественного признака и бинарного таргета.

In [ ]:
sample_size = 3_000  # Размер выборки

# Признаки
X = np.random.uniform(low=-4, high=4, size=(sample_size, 1))

# Таргет
y_mean_true = 1 / (1 + np.exp(1 - 2 * X.ravel()))
y = np.random.binomial(n=1, p=y_mean_true)

plt.figure(figsize=(10, 3))
plt.scatter(X, y_mean_true, marker=".", s=1, label="Реальные вероятности\n(мы их не знаем)")
plt.scatter(X, y, marker="|", alpha=0.1, label="Данные")
plt.xlabel("Признак")
plt.ylabel("Класс объекта")
plt.legend();

Обучите логистическую регрессию, используя реализацию из `sklearn`, при этом свободный коэффициент должен присутствовать в модели. Укажите также `penalty='none'`.

Напечатайте оценку коэффициентов

Ниже объявлена сетка значений признака. По этой сетке постройте
* предсказания классов,
* предсказания вероятностей класса 1.

Визуализируйте эти предсказания. На график стоит нанести также обучающую выборку.

In [ ]:
X_grid = np.linspace(-4, 4, 10_000).reshape((-1, 1))
...

Разбейте отрезок $[-4, 4]$ на одинаковые бины длины длины 0.2 и посчитайте в каждом бине долю объектов класса 1. Полученные значения добавьте на график предсказаний вероятностей и сравните эти графики. Проинтерпретируйте полученные результаты.


<br/><details>
<summary> ➡️ Кликни для показа подсказки </summary>
Может помочь <code>np.digitize</code> и метод <code>groupby</code> для таблиц <code>pandas</code>. Рекомендуем посмотреть <a href="https://thetahat.ru/courses/python">обучающие ноутбуки</a> по библиотекам.
</details>

Позволяют ли посчитанные выше значения построить калибровочную кривую? Если да, поясните и постройте ее график. Если нет, поясните разницу, реализуйте самостоятельно калибровочную кривую и постройте ее график.

<br/><details>
<summary> ➡️ Кликни для показа подсказки </summary>
Посмотри на определение калибровочной кривой и на бины.
</details>

Повторите проведенное исследование для следующих данных и сравните результаты.

In [ ]:
# Признаки
X = np.random.uniform(low=-4, high=4, size=(sample_size, 1))

# Таргет
y_mean_true = 1 / (1 + np.exp(-100 * X.ravel()))
y = np.random.binomial(n=1, p=y_mean_true)

plt.figure(figsize=(10, 3))
plt.scatter(X, y_mean_true, marker=".", s=1, label="Реальные вероятности")
plt.scatter(X, y, marker="|", alpha=0.1, label="Данные")
plt.xlabel("Признак")
plt.ylabel("Класс объекта")
plt.legend();

**Выводы:**

...

---
### Задача 3.

Продолжим исследовать модель логистической регрессии. Сгенерируем данные, состоящие из двух бинарных признаков и бинарного таргета

In [ ]:
probs = np.random.uniform(size=8)
probs /= probs.sum()
probs

x = np.random.choice(np.arange(8), p=probs, size=10_000)
data = pd.DataFrame(
    np.unpackbits(np.array(x.reshape(-1, 1), dtype=">i8").view(np.uint8), axis=1)[:, -3:],
    columns=["feature_1", "feature_2", "target"],
)
data.head()

Особенность таких данных &mdash; конечное число *возможных различных* объектов. В данном случае их всего 4, по количеству всех возможных комбинаций значений признака. Соответственно, любой моделью мы можем сделать только 4 *различных* предсказания. Исследуем, как с этим справляется логистическая регрессия.

Сначала для сравнения посчитайте долю класса 1 для каждой категории объектов.

*Подсказка:* используйте `pd.pivot_table`. Рекомендуем посмотреть <a href="https://thetahat.ru/courses/python">обучающие ноутбуки</a> по библиотекам.

Обучите логистическую регрессию с `penalty='none'` и получите предсказания вероятностей для этих четырех типов объектов. Представьте результаты в таком виде, чтобы их удобно было сравнивать с частотами, посчитанными ранее.

Почему результаты не совпадают?

Для ответа на этот вопрос распишите формулу, которая задает модель логистической регрессии, указав все параметры. Какое предположение о данных при этом делает логистическая регрессия?

...

Предложите и реализуйте способ модификации модели логистической регрессии так, чтобы она точнее предсказывала частоты, посчитанные ранее.

<br/><details>
<summary> ➡️ Кликни для показа подсказки </summary>
Подумайте, какое преобразование признаков можно было бы сделать.
</details>

Опишите ваше предложение:

...

Реализация:

**Выводы:**

...

---
### Задача 4.

*Не все в жизни ограничивается двумя классами, бывает и многоклассовый случай.*

**1.** Пусть $X_1,...,X_n$ &mdash; выборка независимых одинаково распределенных случайных величин из категориального распределения, то есть $\mathsf{P}_\theta(X_1 = j) = \theta_j$ для $j \in \{1, ..., k\}$, причем $\theta = (\theta_1, ..., \theta_k), \theta_j \geqslant 0$ и $\theta_1 + ... + \theta_k = 1$. Найдите оценку максимального правдоподобия параметра $\theta$. Является ли она несмещенной оценкой $\theta$?

<br/><details>
<summary> ➡️ Кликни для показа подсказки </summary>
Посмотрите на пример с лекции для бернуллиевского распределения.
</details>

**2.** Рассмотрим модель логистической регрессии для случая многоклассовой классификации. Пусть метка класса принимает значения в $K$-элементном множестве $\mathscr{Y} = \{1, ..., K\}$. Параметры модели $\theta$ являются матрицей размерности $K \times d$, где $d$ &mdash; количество признаков.

Введем soft-max функцию:

$$
\sigma(z) = \left( \frac{e^{z_1}}{\sum_{i=1}^K e^{z_i}}, \dots, \frac{e^{z_K}}{\sum_{i=1}^K e^{z_i}} \right).
$$

Из определения ясно, что $\sum\limits_{i=1}^k \sigma_k(z) = 1$. Название функции связано с тем, что самая большая компонента вектора $z$ будет близка к $1$, а все остальные будут малы, но не равны нулю. Таким образом, происходит сглаженное взятие `argmax`.

По аналогии с обычной логистической регрессией рассмотрим выборку объектов $x_1, ...,  x_n$, где $x_i = (x_{i1}, \dots, x_{id})^T \in \mathscr{X}$, и выборку таргетов $Y_1, ...,  Y_n \in \mathscr{Y}$

Модель логистической регрессии в многоклассовом случае предполагает, что $\mathsf{P}_{\theta}(Y_{ik} = k) = \sigma_k(\theta x)$.

Выполните следующие действия.

1. Определите множество моделей $\mathcal{M}$, соответствующее логистической регрессии.

2. Запишите функцию правдоподобия для такой модели.

3. Выпишите формулы GD и SGD для максимизации этой функции правдоподобия. В формулу нужно упростить подобно тому, как было показано на занятии.

*Логистическую регрессию для многоклассового случая можно применять и другими способами, подробнее мы разберем на следующих занятиях.*

---
### Задача 5.

*Устали писать формулы? Самое время что-то закодить!*

*Кроме кодинга в задаче есть исследование. Там, где это возможно, вы можете выполнить исследование без реализации и получить за это частичные баллы.*


**1.** Реализуйте логистическую регрессию для двух вариантов поиска оценки параметров:
* простой градиентный спуск;
* стохастический градиентный спуск с `batch_size` элементами на каждой итерации.

Останавливайте итерации при выполнении хотя бы одного из двух условий:
* количество итераций превзошло число `max_iter`;
* оптимизируемый функционал изменился за итерацию не более чем на `tol`.

При выполнении каждой итерации с целью дальнейшего анализа сохраняйте текущее значение оптимизируемого функционала, а также затраченное время на итерацию.  **При реализации класса запрещено пользоваться ИИ-инструментами.**

*Замечания.*

1. Для чистоты эксперимента время шага внутри цикла нужно замерять от конца предыдущего шага до конца текущего, а не от начала текущего шага. Время измеряйте с помощью `from time import time`.

2. Иногда при подсчете сигмоиды и оптимизируемого функционала могут возникать вычислительные ошибки. Для их избежания существуют специальные трюки.
    * [How to Evaluate the Logistic Loss and not NaN trying](http://fa.bianp.net/blog/2019/evaluate_logistic/)
    * [Exp-normalize trick](https://timvieira.github.io/blog/post/2014/02/11/exp-normalize-trick/)<br>
3. Трюки не обязательно реализовывать самостоятельно, можете воспользоваться функциями для них из `numpy` или `scipy`:
    * [`numpy.logaddexp`](https://numpy.org/doc/stable/reference/generated/numpy.logaddexp.html);
    * [`scipy.special.logsumexp`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.logsumexp.html).
4. Обратите внимание, что класс `LogisticRegression` &mdash; наследник класса `BaseEstimator`, это с легкостью позволит использовать наш класс в различных пайплайнах библиотеки `sklearn`.
4. Следите за качеством кода, комментируйте логические этапы кода. Несоблюдение этого требования может привести к потере баллов.


In [ ]:
from sklearn.base import BaseEstimator
import numpy as np
from time import time
from typing import Literal
from scipy.special import expit, logsumexp


class LogisticRegression(BaseEstimator):
    """Модель логистической регрессии.

    Параметры:
    method (Literal['gd', 'sgd']): Метод оптимизации ('gd' - градиентный спуск,
        'sgd' - стохастический градиентный спуск).
    learning_rate (float): Константа скорости обучения, на которую домножаем градиент при обучении
    tol (float): Допустимое изменение функционала между итерациями.
    max_iter (int): Максимальное число итераций.
    batch_size (int): Размер выборки для оценки градиента (используется только при 'sgd').
    fit_intercept (bool): Добавлять ли константу в признаки.
    save_history (bool): Сохранять ли историю обучения.
    """

    def __init__(
        self,
        method: Literal["gd", "sgd"] = "gd",
        learning_rate: float = 0.5,
        tol: float = 1e-3,
        max_iter: int = int(1e4),
        batch_size: int = 64,
        fit_intercept: bool = True,
        save_history: bool = True,
    ):
        """Создает модель и инициализирует параметры."""
        self.method = method
        self.learning_rate = learning_rate
        self.tol = tol
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.fit_intercept = fit_intercept
        self.save_history = save_history
        self.history = []  # История обучения

    @staticmethod
    def _sigmoid(x: np.ndarray) -> np.ndarray:
        """Вычисляет сигмоидную функцию."""
        # Use logaddexp for numerical stability
        return expit(x)

    def _log_loss(self, X: np.ndarray, Y: np.ndarray, coef: np.ndarray) -> float:
        """Вычисляет значение логистической функции потерь."""
        scores = X @ coef
        # Use logsumexp for numerical stability
        log_likelihood = np.sum(Y * logsumexp(0, scores) - logsumexp(0, scores))
        return -log_likelihood / X.shape[0]


    def _add_intercept(self, X: np.ndarray) -> np.ndarray:
        """Добавляет свободный коэффициент к матрице признаков.

        Параметры: X (np.ndarray): Исходная матрица признаков.

        Возвращает: np.ndarray: Матрица X с добавленным свободным
        коэффициентом.
        """
        X_copy = np.full((X.shape[0], X.shape[1] + 1), fill_value=1.0)
        X_copy[:, :-1] = X
        return X_copy

    def fit(self, X: np.ndarray, Y: np.ndarray) -> "LogisticRegression":
        """Обучает модель логистической регрессии.

        Также, в случае self.save_history=True, добавляет в self.history
        текущее значение оптимизируемого функционала и затраченное время.

        Параметры:
        X (np.ndarray): Матрица признаков.
        Y (np.ndarray): Вектор истинных меток.

        Возвращает:
        LogisticRegression: Обученная модель.
        """
        if X.shape[0] != Y.shape[0]:
            raise ValueError("Количество строк в X и Y должно совпадать")

        if self.fit_intercept:
            X_copy = self._add_intercept(X)
        else:
            X_copy = X.copy()

        # Шаг спуска
        n_samples, n_features = X_copy.shape
        coef = np.zeros(n_features).reshape(-1, 1)
        car_sec = time()
        prev_loss = np.inf  # Initialize with infinity for the first iteration


        if self.method == "gd":
          # Градиентный спуск

          for i in range(self.max_iter):

            scores = X_copy @ coef
            predictions = self._sigmoid(scores)
            grad = X_copy.T @ (predictions - Y.reshape(-1, 1)) / n_samples
            new_coef = coef - self.learning_rate * grad
            new_loss = self._log_loss(X_copy, Y, new_coef)


            if abs(new_loss - prev_loss) < self.tol:
              break

            coef = new_coef
            prev_loss = new_loss
            new_sec = time()

            if self.save_history:
                self.history.append([new_loss, new_sec - car_sec]) # Сохраняем в историю значение текущее значение функционала и время работы шага

            car_sec = new_sec


        elif self.method == "sgd":
          # Стохастический градиентный спуск

          for i in range(self.max_iter):

            sample_indices = np.random.randint(n_samples, size=self.batch_size)
            X_sample = X_copy[sample_indices]
            Y_sample = Y[sample_indices].reshape(-1, 1)

            scores = X_sample @ coef
            predictions = self._sigmoid(scores)
            grad = X_sample.T @ (predictions - Y_sample) / self.batch_size


            new_coef = coef - self.learning_rate * grad
            # Recalculate likelihood over the whole dataset for convergence check
            new_loss = self._log_loss(X_copy, Y, new_coef)


            if abs(new_loss - prev_loss) < self.tol: # Check tolerance after the first iteration
              break

            coef = new_coef
            prev_loss = new_loss
            new_sec = time()

            if self.save_history:
                self.history.append([new_loss, new_sec - car_sec]) # Сохраняем в историю значение текущее значение функционала и время работы шага

            car_sec = new_sec


        self.coef_ = coef  # Коэффициенты модели
        self.intercept_ = coef[0] if self.fit_intercept else 0 # Свободный коэффициент
        self.n_iter_ = i + 1 # Число итераций

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Возвращает предсказанные классы.

        Параметры: X (np.ndarray): Матрица признаков.

        Возвращает: np.ndarray: Предсказанные классы.
        """
        prob_class_1 = self.predict_proba(X)[:, 1]
        predictions = (prob_class_1 >= 0.5).astype(int)

        return predictions


    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """Возвращает вероятности классов 0 и 1.

        Параметры: X (np.ndarray): Матрица признаков.

        Возвращает: np.ndarray: Матрица вероятностей классов (n_samples,
        2).
        """
        if self.fit_intercept:
            X_copy = self._add_intercept(X)
        else:
            X_copy = X.copy()

        if X_copy.shape[1] != self.coef_.shape[0]:
            raise ValueError("Число признаков в X не соответствует числу коэффициентов модели")

        prob_class_1 = self._sigmoid(X_copy @ self.coef_)
        prob_predictions = np.hstack([1 - prob_class_1, prob_class_1])

        return prob_predictions

In [ ]:
from sklearn.base import BaseEstimator
import numpy as np
from time import time
from typing import Literal
from scipy import special


class LogisticRegression(BaseEstimator):
    """Модель логистической регрессии.

    Параметры:
    method (Literal['gd', 'sgd']): Метод оптимизации ('gd' - градиентный спуск,
        'sgd' - стохастический градиентный спуск).
    learning_rate (float): Константа скорости обучения, на которую домножаем градиент при обучении
    tol (float): Допустимое изменение функционала между итерациями.
    max_iter (int): Максимальное число итераций.
    batch_size (int): Размер выборки для оценки градиента (используется только при 'sgd').
    fit_intercept (bool): Добавлять ли константу в признаки.
    save_history (bool): Сохранять ли историю обучения.
    """

    def __init__(
        self,
        method: Literal["gd", "sgd"] = "gd",
        learning_rate: float = 0.5,
        tol: float = 1e-3,
        max_iter: int = int(1e4),
        batch_size: int = 64,
        fit_intercept: bool = True,
        save_history: bool = True,
    ):
        """Создает модель и инициализирует параметры."""
        self.method = method
        self.learning_rate = learning_rate
        self.tol = tol
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.fit_intercept = fit_intercept
        self.save_history = save_history
        self.history = []  # История обучения

    @staticmethod
    def _sigmoid(x: np.ndarray) -> np.ndarray:
        """Вычисляет сигмоидную функцию."""
        # Use logaddexp for numerical stability
        return special.expit(x)


    def _log_sigmoid(self, x: np.ndarray) -> np.ndarray:
        """Вычисляет логарифм сигмоидной функции"""
        # Use logaddexp for numerical stability
        return -np.logaddexp(0, -x)

    def _add_intercept(self, X: np.ndarray) -> np.ndarray:
        """Добавляет свободный коэффициент к матрице признаков.

        Параметры: X (np.ndarray): Исходная матрица признаков.

        Возвращает: np.ndarray: Матрица X с добавленным свободным
        коэффициентом.
        """
        X_copy = np.full((X.shape[0], X.shape[1] + 1), fill_value=1.0)
        X_copy[:, :-1] = X
        return X_copy

    def fit(self, X: np.ndarray, Y: np.ndarray) -> "LogisticRegression":
        """Обучает модель логистической регрессии.

        Также, в случае self.save_history=True, добавляет в self.history
        текущее значение оптимизируемого функционала и затраченное время.

        Параметры:
        X (np.ndarray): Матрица признаков.
        Y (np.ndarray): Вектор истинных меток.

        Возвращает:
        LogisticRegression: Обученная модель.
        """
        if X.shape[0] != Y.shape[0]:
            raise ValueError("Количество строк в X и Y должно совпадать")

        if self.fit_intercept:
            X_copy = self._add_intercept(X)
        else:
            X_copy = X.copy()
        Y_copy = Y.reshape(-1, 1) # Ensure Y is a column vector


        # Шаг спуска

        coef = np.zeros(X_copy.shape[1]).reshape(-1, 1)
        car_sec = time()
        car_likelyhood = -np.inf  # Initialize with negative infinity for the first iteration

        if self.method == "gd":
          # Градиентный спуск

          for _ in range(self.max_iter):

            grad = X_copy.T @ (Y_copy - self._sigmoid(X_copy @ coef))
            new_coef = coef + self.learning_rate * grad
            new_likelyhood = np.sum(Y_copy * self._log_sigmoid(X_copy @ new_coef) + (1-Y_copy)*self._log_sigmoid(- X_copy @ new_coef))


            if (_ > 0) and (abs(new_likelyhood - car_likelyhood) < self.tol):
              break

            coef = new_coef
            car_likelyhood = new_likelyhood
            new_sec = time()

            if self.save_history:
                self.history.append([car_likelyhood, new_sec - car_sec]) # Сохраняем в историю значение текущее значение функционала и время работы шага

            car_sec = new_sec


        elif self.method == "sgd":
          # Стохастический градиентный спуск

          for _ in range(self.max_iter):

            sample_indices = np.random.randint(X_copy.shape[0], size=self.batch_size)
            X_sample = X_copy[sample_indices]
            Y_sample = Y_copy[sample_indices]

            grad = X_sample.T @ (Y_sample - self._sigmoid(X_sample @ coef)) # Removed the scaling factor


            new_coef = coef + self.learning_rate * grad
            # Recalculate likelihood over the whole dataset for convergence check
            new_likelyhood = np.sum(Y_copy * self._log_sigmoid(X_copy @ new_coef) + (1-Y_copy)*self._log_sigmoid(- X_copy @ new_coef))


            if (_ > 0) and (abs(new_likelyhood - car_likelyhood) < self.tol): # Check tolerance after the first iteration
              break

            coef = new_coef
            car_likelyhood = new_likelyhood
            new_sec = time()

            if self.save_history:
                self.history.append([car_likelyhood, new_sec - car_sec]) # Сохраняем в историю значение текущее значение функционала и время работы шага

            car_sec = new_sec


        self.coef_ = coef  # Коэффициенты модели
        self.intercept_ = coef[0] if self.fit_intercept else 0 # Свободный коэффициент
        self.n_iter_ = _ + 1 # Число итераций

        return self

    def predict(self, X: np.ndarray) -> np.ndarray:
        """Возвращает предсказанные классы.

        Параметры: X (np.ndarray): Матрица признаков.

        Возвращает: np.ndarray: Предсказанные классы.
        """
        prob_class_1 = self.predict_proba(X)[:, 1]
        predictions = (prob_class_1 >= 0.5).astype(int)

        return predictions


    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        """Возвращает вероятности классов 0 и 1.

        Параметры: X (np.ndarray): Матрица признаков.

        Возвращает: np.ndarray: Матрица вероятностей классов (n_samples,
        2).
        """
        if self.fit_intercept:
            X_copy = self._add_intercept(X)
        else:
            X_copy = X.copy()

        if X_copy.shape[1] != self.coef_.shape[0]:
            raise ValueError("Число признаков в X не соответствует числу коэффициентов модели")

        prob_class_1 = self._sigmoid(X_copy @ self.coef_)
        prob_predictions = np.hstack([1 - prob_class_1, prob_class_1])

        return prob_predictions

Рассмотрим датасет [Diabetes Health Indicators](https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset).

**Для данного задания будем рассматривать версию датасета** `diabetes_binary_5050split_health_indicators_BRFSS2015.csv`


Этот датасет содержит статистику здравоохранения и информацию об образе жизни, полученную в результате опросов вместе с меткой наличия/отсутствия диабета у участников. Среди признаков есть демографические данные, результаты лабораторных тестов и ответы на вопросы анкеты. Целевая переменная  `Diabetes_binary` определяет статус пациента: есть ли у него диабет или предиабет (`1`), или он здоров (`0`).



Рассмотрим некоторые признаки, представленные в датасете.

**Показатели здоровья**

- `HighBP`: Высокое кровяное давление (`1` = да, `0` = нет).

- `HighChol`: Высокий уровень холестерина (`1` = да, `0` = нет).

- `CholCheck`: Проверка уровня холестерина за последние 5 лет (`1` = да, `0` = нет).

- `BMI`: Индекс массы тела (рассчитывается как вес (кг) / рост² (м²)).

- `GenHlth`: Общая оценка здоровья (`1` = отличное, `2` = очень хорошее, ..., `5` = плохое).

**Образ жизни**
- `Smoker`: Статус курения (`1` = выкурил ≥100 сигарет за жизнь, `0` = нет).

- `PhysActivity`: Физическая активность вне работы (`1` = да, `0` = нет).

- `Fruits`: Регулярное употребление фруктов (`1` = не менее 1 раз в день, `0` = реже).

**Доступ к медицине**
- `AnyHealthcare`: Наличие медицинской страховки (`1` = да, `0` = нет).

- `NoDocbcCost`: Отказ от визита к врачу из-за стоимости (`1` = да, `0` = нет).



Скачайте файл и прочитайте его с помощью `pandas`.

In [ ]:
dataset = pd.read_csv("diabets_health_indicators.csv")
dataset.head()

Разделите выборку на обучающую и тестовую и выполните преобразование категориальных признаков.

Для интерпретации коэффициентов необходимо нормализовать данные. Воспользуемся для этого классом `StandardScaler` из библиотеки `sklearn`.

In [ ]:
scaler = StandardScaler()
...

**2.** Обучите две модели логистической регрессии с помощью методов
* простой градиентный спуск;
* стохастический градиентный спуск.

Постройте график, на котором нанесите две кривые обучения, каждая из которых отображает зависимость оптимизируемого функционала от номера итерации метода. **Функционал должен быть одинаковый для всех моделей**. Нарисуйте также график зависимости этого функционала от времени работы метода.

*Замечания:*
* Все графики должны быть информативны, с подписанными осями и т.д..
* Для чистоты эксперимента желательно не запускать в момент обучения другие задачи и провести обучение несколько раз, усреднив результаты.

Сделайте выводы. Что будет при обучении на датасете, если  увеличить количество объектов, а число признаков оставить прежним?

...

**3.** Исследуйте влияние размер шага (`learning_rate`) на качество модели для двух режимов обучения (простой и стохастический градиентный спуск). Для каждого размера шага получите качество модели при использовании простого и стохастического градиентного спуска. Сравните качество полученных моделей по метрике `accuracy`.

In [ ]:
learning_rate_list = np.logspace(-5, 3, 8)

Сделайте выводы

...

Постройте кривые обучения для различных `learning_rate`. Не обязательно рассматривать все `learning_rate`, так как их слишком много, и график будет нагроможден. Возьмите около половины из них.

Какой `learning_rate` стоит выбирать в зависимости от способа обучения модели? Чем плохи маленькие и большие `learning_rate`?

...

**4.** Рассмотрите наилучшую модель с предыдущего шага. Визуализируйте значения полученных коэффициентов.

Как можно проинтерпретировать полученные результаты относительно решаемой задачи?

...

**5.** Сравните данную модель с бейзлайном, который в качестве предсказания выдает самый частый класс на обучающей выборке.

Насколько хорошее получилось качество обученной модели?

...

**Вывод:**

...

---
© 2025 команда <a href="https://thetahat.ru/">ThetaHat</a> для курса ML-1 ШАД